In [1]:
import cv2
import numpy as np
import os
import tensorflow as tf

2025-10-06 00:18:11.308931: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-06 00:18:11.459473: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-06 00:18:12.293807: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [25]:
image_path = os.getcwd() + "/data/FER-2013"
img_size = (48,48)
batch_size = 32

In [26]:
train = tf.keras.utils.image_dataset_from_directory(image_path + "/train",
                                                    seed = 0,
                                                    image_size = img_size,
                                                    subset = 'training',
                                                    validation_split = 0.2,
                                                    color_mode = "grayscale",
                                                    batch_size = batch_size)

val = tf.keras.utils.image_dataset_from_directory(image_path + "/train",
                                                    seed = 0,
                                                    image_size = img_size,
                                                    subset = 'validation',
                                                    validation_split = 0.2,
                                                    color_mode = "grayscale",
                                                    batch_size = batch_size)

test = tf.keras.utils.image_dataset_from_directory(image_path + "/test",
                                                    image_size = img_size,
                                                    color_mode = "grayscale",
                                                    batch_size = batch_size)

Found 28709 files belonging to 7 classes.
Using 22968 files for training.
Found 28709 files belonging to 7 classes.
Using 5741 files for validation.
Found 7178 files belonging to 7 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(48,48,1)),
    tf.keras.layers.Conv2D(filters=64,
                           kernel_size = (3,3),
                           activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(filters=64,
                           kernel_size = (3,3),
                           activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])

In [30]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
history = model.fit(
    train,
    validation_data = test,
    epochs = 50
    # callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)]
)

Epoch 1/50
718/718 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.4677 - loss: 1.3823 - val_accuracy: 0.4408 - val_loss: 1.4560
Epoch 2/50
718/718 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.5296 - loss: 1.2315 - val_accuracy: 0.4478 - val_loss: 1.4482
Epoch 3/50
718/718 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.5917 - loss: 1.0871 - val_accuracy: 0.4649 - val_loss: 1.5781
Epoch 4/50
718/718 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.6675 - loss: 0.9046 - val_accuracy: 0.4671 - val_loss: 1.6645
Epoch 5/50
718/718 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.7419 - loss: 0.7206 - val_accuracy: 0.4494 - val_loss: 2.0157
Epoch 6/50
718/718 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.8120 - loss: 0.5450 - val_accuracy: 0.4815 - val_loss: 1.9955
Epoch 7/50
718/718 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.8687 - loss: 0.3939 - val_accuracy: 0.4794 - val_loss: 2.2408
Epoch 8/50
718/718 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9059 - loss: 0.2869 - val_accu

In [ ]:
model.save("model_fer.keras")

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=model_fer.